# MNIST image classification with Keras Tensorflow backend

In [1]:
import os
import shutil
from datetime import datetime
import sys
import json
import argparse

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

2.1.0


In [17]:
PROJECT = "ml-practice-260405" # REPLACE WITH YOUR PROJECT ID
BUCKET = "bucket-ml-practice-260405" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "cnn"  # "linear", "dnn", "dnn_dropout", or "cnn"

In [18]:
# Do not change 
os.environ["ACCOUNT"] = "sandcorp2014@gmail.com"
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "1.13"  # Tensorflow version

In [4]:
%%bash
gcloud config set account $ACCOUNT

Updated property [core/account].


In [5]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [6]:
%%bash
mkdir mnistmodel_keras_tf
mkdir mnistmodel_keras_tf/trainer/

In [7]:
%%writefile mnistmodel_keras_tf/trainer/__init__.py
##

Writing mnistmodel_keras_tf/trainer/__init__.py


In [11]:
%%writefile mnistmodel_keras_tf/trainer/task.py
import argparse
import json
import os

from .import model
import tensorflow as tf

if __name__ == '__main__':
    # Create a parser object
    parser = argparse.ArgumentParser()
    
    # Input argument in the parser object
    parser.add_argument(
        "--train_batch_size",
        help="Batch size for training",
        type = int,
        default=100
    )
    
    parser.add_argument(
        "--learning_rate",
        help="Initial Learning Rate For Training",
        type=float,
        default=0.01
    )
    
    parser.add_argument(
        "--train_steps",
        help="Steps to run the training jobs for",
        type=int,
        default=0
    )
    
    parser.add_argument(
        "--output_dir",
        help="Local Or GCS location to write checkpoint and export model",
        required=True
    )
    
    # Generate List of models to print in help message
    model_names = [name.replace("_model","") for name in dir(model) if name.endswith("_model")]
    
    parser.add_argument(
        '--model',
        help="Type of model. Supported types are {}".format(model_names),
        default='cnn'
    )
    
    parser.add_argument(
        "--job-dir",
        help = "this model ignores this field, but it is required by gcloud",
        default = "junk"
    )
    
    # Optional Hyperparameters used by cnn
    parser.add_argument(
        '--ksize1',
        help = "Kernel size of the first layer of the cnn",
        type=int,
        default=5
    )
    
    parser.add_argument(
        '--ksize2',
        help = "Kernel size of the second layer of the cnn",
        type=int,
        default=5
    )
    
    parser.add_argument(
        '--nfil1',
        help = "Number of filters in the first layer of the cnn",
        type=int,
        default=10
    )
    
    parser.add_argument(
        '--nfil2',
        help = "Number of filters in the second layer of the cnn",
        type=int,
        default=20
    )
    
    parser.add_argument(
        '--dprob',
        help ='Drop out probability for cnn',
        type = float,
        default=0.25
    )
    
    parser.add_argument(
        '--batch_norm',
        help = "If specified, do batch_norm for cnn",
        dest = "batch_norm",
        action = "store_true"
    )
    parser.set_defaults(batch_norm = False)
    
    args = parser.parse_args()
    hparams = args.__dict__

    # unused args provided by service
    hparams.pop("job_dir", None)
    hparams.pop("job-dir", None)

    output_dir = hparams.pop("output_dir")
    # Append trial_id to path so hptuning jobs don"t overwrite eachother
    output_dir = os.path.join(
        output_dir,
        json.loads(
            os.environ.get("TF_CONFIG", "{}")
        ).get("task", {}).get("trial", "")
    )
    
    # Calculate train_steps if not provided
    if hparams["train_steps"] < 1:
        # 10,000 steps with batch_size of 512
        hparams["train_steps"] = (10000 * 512) // hparams["train_batch_size"]
        print("Training for {} steps.".format(hparams["train_steps"]))
    
    # Run the training job
    model.train_and_evaluate(output_dir, hparams)

Overwriting mnistmodel_keras_tf/trainer/task.py


In [12]:
%%writefile mnistmodel_keras_tf/trainer/model.py
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

HEIGHT=28
WIDTH=28
NCLASSES=10

# Build Keras model using Keras Sequential API
def linear_model(hparams):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 1], name="image"))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=tf.nn.softmax,
        name="probabilities"
    ))
    return model
    
def dnn_model(hparams):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 1], name="image"))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=300, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=100, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=30, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=NCLASSES, activation=tf.nn.softmax, name='Probabilities'))
    return model

def dnn_dropout_model(hparams):
    dprob = hparams.get("dprob", 0.1)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=[HEIGHT, WIDTH, 1], name="image"))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=300, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=100, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(units=30, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(rate=dprob))
    model.add(tf.keras.layers.Dense(
        units=NCLASSES, 
        activation=tf.nn.softmax,
        name="probabilities"
    ))
    return model

def cnn_model(hparams):
    ksize1 = hparams.get('ksize1', 5)
    ksize2 = hparams.get('ksize2', 5)
    nfil1 = hparams.get('nfil1', 10)
    nfil2 = hparams.get('nfil2', 20)
    dprob = hparams.get('dprob', 0.25)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(
        input_shape = [HEIGHT, WIDTH, 1],
        name = "image"
    )) # shape = (?, 28, 28, 1)
    model.add(tf.keras.layers.Conv2D(
        filters=nfil1,
        kernel_size=ksize1,
        padding='same',
        activation=tf.nn.relu
    )) # shape = (?, 28, 28, nfil1)
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=2,
        strides=2
    )) # shape = (?, 14, 14, nfil1)
    model.add(tf.keras.layers.Conv2D(
        filters=nfil2,
        kernel_size=ksize2,
        padding='same',
        activation=tf.nn.relu
    )) # shape = (?, 14, 14, nfil2)
    model.add(tf.keras.layers.MaxPooling2D(
        pool_size=2,
        strides=2
    )) # shape = (?, 7, 7, nfil2)
    model.add(tf.keras.layers.Flatten())
    
    # Apply Batch Normalization
    if hparams['batch_norm']:
        model.add(tf.keras.layers.Dense(
            units=300,
            activation=tf.nn.relu
        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Activation(
            activation=tf.nn.relu
        ))
    else:
        model.add(tf.keras.layers.Dense(
            units=300,
            activation=tf.nn.relu
        ))
        
    # Apply Drop Out
    model.add(tf.keras.layers.Dropout(rate=dprob))
    
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=None
    ))
    
    # Apply Batch Normalization Once More
    if hparams['batch_norm']:
        model.add(tf.keras.layers.BatchNormalization())
        
    # SoftMax Layer
    model.add(tf.keras.layers.Dense(
        units=NCLASSES,
        activation=tf.nn.softmax,
        name='probabilities'
    ))
    
    return model

# Create serving input function for inference
def serving_input_fn():
    # Input will be rank 3
    feature_placeholders = {
        "image": tf.compat.v1.placeholder(dtype = tf.float32, shape=[None, HEIGHT, WIDTH])
    }
    # But model function require rank 4
    features = {
        "image": tf.expand_dims(input=feature_placeholders["image"], axis=-1)
    }
    return tf.estimator.export.ServingInputReceiver(
        features=features,
        receiver_tensors=feature_placeholders
    )

    
# Create train and evaluate function
def train_and_evaluate(ouput_dir, hparams):
    # Ensure filewriter cache is clear for TensorBoard event file.
    tf.compat.v1.summary.FileWriterCache.clear()
    
    EVAL_INTERVAL = 60
    
    # Get mnist Data
    mnist = tf.keras.datasets.mnist
    
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    # Scale our training and testing features between 0 and 1
    x_train, x_test = x_train / 255.0, x_test / 255.0
    
    # Reshape image to add a dimension for channels (1 in this case)
    x_train = x_train.reshape([-1, HEIGHT, WIDTH, 1])
    x_test = x_test.reshape([-1, HEIGHT, WIDTH, 1])
    
    # Convert labels to categorical one hot encoding
    y_train = tf.keras.utils.to_categorical(y=y_train, num_classes=NCLASSES)
    y_test = tf.keras.utils.to_categorical(y=y_test, num_classes=NCLASSES)
    
    # Create training input function
    train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"image": x_train},
        y=y_train,
        batch_size=100,
        num_epochs=None,
        shuffle=True,
        queue_capacity=5000
    )
    
    # Create evaluation input function
    eval_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
        x={"image": x_test},
        y=y_test,
        batch_size=100,
        num_epochs=1,
        shuffle=False,
        queue_capacity=5000
    )
    
    # Build Keras model
    model = cnn_model(hparams=hparams)
    
    # Compile Keras model with optimizer, loss function and eval metric
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Convert Keras model to estimator
    estimator = tf.keras.estimator.model_to_estimator(
        keras_model=model,
        model_dir=ouput_dir,
        config=tf.estimator.RunConfig(save_checkpoints_secs=EVAL_INTERVAL)
    )
    
    # Set estimator's train_spec to use train_input_fn() and train for so many steps
    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn,
        max_steps=hparams['train_steps']
    )
    
    # Create exporter that use serving_input_fn() to 
    # create saved_model for serving.
    exporter = tf.estimator.LatestExporter(
        name='exporter',
        serving_input_receiver_fn=serving_input_fn
    )
    
    # Set estimators eval_spec to use eval_input_fn() and 
    # export saved_model
    eval_spec = tf.estimator.EvalSpec(
        input_fn=eval_input_fn,
        steps=None,
        exporters=exporter,
        throttle_secs=EVAL_INTERVAL
    )
    
    # Run train_and_evaluate loop
    tf.estimator.train_and_evaluate(
        estimator=estimator,
        train_spec=train_spec,
        eval_spec=eval_spec
    )

Overwriting mnistmodel_keras_tf/trainer/model.py


## Run as a Python module

Since we want to run our code on Cloud ML Engine, we've packaged it as a python module.

The `model.py` and `task.py` containing the model code is in <a href="mnistmodel_keras_tf/trainer">mnistmodel_keras_tf/trainer</a>

**Let's first run it locally for a few steps to test the code.** 

In [13]:
%%bash
rm -rf mnistmodel_keras_tf.tar.gz mnist_keras_tf_trained
JOB_DIR=./tmp
gcloud ml-engine local train \
    --module-name=trainer.task \
    --package-path=./mnistmodel_keras_tf/trainer \
    --job-dir=JOB_DIR \
    -- \
    --output_dir=./mnist_keras_tf_trained \
    --train_steps=100 \
    --learning_rate=0.01

2020-08-05 10:05:14.310346: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-08-05 10:05:14.310414: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-08-05 10:05:14.310422: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
W0805 10:05:15.341052 140584698599232 module_wrapper.py:138] From /media/mujahid7292/Data/GoogleDriveSandCorp2014/ML_With_TensorFlow_On_GCP/08.Image Understanding with TensorFlow on GCP/Week_1/Lab_4_Image_Classification_with_a_CNN_Model/Practice/mnistmodel_keras_tf/trainer/model.p

<b><p style='color:red'>Loss for final step: 0.4481694</p></b>

Run as python module.

In [14]:
%%bash
rm -rf mnistmodel_keras_tf.tar.gz mnist_keras_tf_trained
python3 -m mnistmodel_keras_tf.trainer.task \
    --learning_rate=0.01 \
    --train_steps=100 \
    --output_dir=./mnist_keras_tf_trained

2020-08-05 10:05:50.472670: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-08-05 10:05:50.472744: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-08-05 10:05:50.472752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
W0805 10:05:51.539287 140438488995648 module_wrapper.py:138] From /media/mujahid7292/Data/GoogleDriveSandCorp2014/ML_With_TensorFlow_On_GCP/08.Image Understanding with TensorFlow on GCP/Week_1/Lab_4_Image_Classification_with_a_CNN_Model/Practice/mnistmodel_keras_tf/trainer/model.p

<b><p style='color:red'>Loss for final step: 0.42073974</p></b>

**Now, let's do it on Cloud ML Engine so we can train on GPU (`--scale-tier=BASIC_GPU`)**

Note the GPU speed up depends on the model type. You'll notice the more complex CNN model trains significantly faster on GPU, however the speed up on the simpler models is not as pronounced.

In [19]:
%%bash
OUTDIR=gs://${BUCKET}/mnist/trained_dnn
JOBNAME=mnist_cnn_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --region=$REGION \
    --module-name=trainer.task \
    --package-path=./mnistmodel_keras_tf/trainer \
    --job-dir=$OUTDIR \
    --staging-bucket=gs://$BUCKET \
    --scale-tier=BASIC_GPU \
    --runtime-version=$TFVERSION \
    -- \
    --output_dir=$OUTDIR \
    --train_steps=100 \
    --learning_rate=0.01 \
    --train_batch_size=512 

gs://bucket-ml-practice-260405/mnist/trained_dnn us-central1 mnist_dnn_200805_040956
jobId: mnist_dnn_200805_040956
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [mnist_dnn_200805_040956] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe mnist_dnn_200805_040956

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs mnist_dnn_200805_040956


<b><p style='color:red'>Loss for final step: 0.5132059</p></b>

## Deploying and predicting with model

Deploy the model:

In [20]:
%%bash
MODEL_NAME="mnist_cnn_dropout"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/mnist/trained_${MODEL_TYPE}/export/exporter | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

Deleting and deploying mnist_dnn_dropout cnn from  ... this will take a few minutes


CommandException: One or more URLs matched no objects.
Created ml engine model [projects/ml-practice-260405/models/mnist_dnn_dropout].
ERROR: (gcloud.ml-engine.versions.create) argument --origin: expected one argument
Usage: gcloud ml-engine versions create VERSION --model=MODEL [optional flags]
  optional flags may be  --async | --config | --description | --framework |
                         --help | --labels | --origin | --python-version |
                         --runtime-version | --staging-bucket

For detailed information on this command and its flags, run:
  gcloud ml-engine versions create --help


CalledProcessError: Command 'b'MODEL_NAME="mnist_dnn_dropout"\nMODEL_VERSION=${MODEL_TYPE}\nMODEL_LOCATION=$(gsutil ls gs://${BUCKET}/mnist/trained_${MODEL_TYPE}/export/exporter | tail -1)\necho "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"\n#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}\n#gcloud ml-engine models delete ${MODEL_NAME}\ngcloud ml-engine models create ${MODEL_NAME} --regions $REGION\ngcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION\n'' returned non-zero exit status 2.

To predict with the model, let's take one of the example images.

In [ ]:
import json, codecs
import matplotlib.pyplot as plt
import tensorflow as tf

HEIGHT = 28
WIDTH = 28

# Get mnist data
mnist = tf.keras.datasets.mnist

(_, _), (x_test, _) = mnist.load_data()

# Scale our features between 0 and 1
x_test = x_test / 255.0 

IMGNO = 5 # CHANGE THIS to get different images
jsondata = {"image": x_test[IMGNO].reshape(HEIGHT, WIDTH).tolist()}
json.dump(jsondata, codecs.open("test.json", 'w', encoding = "utf-8"))
plt.imshow(x_test[IMGNO].reshape(HEIGHT, WIDTH));

Send it to the prediction service

In [ ]:
%%bash
gcloud ai-platform predict \
    --model=mnist_dnn_dropout \
    --version=${MODEL_TYPE} \
    --json-instances=./test.json

## Remove Trained Directory

In [ ]:
%%bash
rm -rf mnistmodel_keras_tf.tar.gz mnist_keras_tf_trained